In [56]:
#Imports e configuração de caminho
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys, os

src_path = os.path.join(os.getcwd(), "src")
sys.path.append(src_path)

from src.utils import (
    normalize_column_names,
    build_iso_date,
    impute_missing_values,
    create_lag_features,
    create_rolling_features
)
# Cria pasta de saída
os.makedirs("data/processed", exist_ok=True)

In [57]:
#Carrega os dados brutos
df_dengue = pd.read_csv("data/raw/dengue.csv")
df_clima  = pd.read_csv("data/raw/clima_inmet.csv")
df_pop    = pd.read_csv("data/raw/pop_density.csv")

df_dengue = normalize_column_names(df_dengue)
df_clima  = normalize_column_names(df_clima)
df_pop    = normalize_column_names(df_pop)

print("Dengue:", df_dengue.shape)
print("Clima:", df_clima.shape)
print("População:", df_pop.shape)

Dengue: (812, 30)
Clima: (313, 6)
População: (1, 2)


In [58]:
#Prepara base de dengue
# Extrai ano e semana epidemiológica
df_dengue["data_inise"] = pd.to_datetime(df_dengue["data_inise"])
df_dengue["year"] = df_dengue["data_inise"].dt.isocalendar().year
df_dengue["week"] = df_dengue["data_inise"].dt.isocalendar().week

df_dengue = df_dengue.groupby(["localidade_id", "year", "week"], as_index=False).agg({
    "casos": "sum", "rt": "mean", "pop": "mean", "tempmed": "mean", "umidmed": "mean"
})


In [59]:
#junta com clima e população
df = df_dengue.merge(df_clima, on=["year", "week"], how="left")
df = df.merge(df_pop, left_on="localidade_id", right_on="municipio_id", how="left")
df = df.infer_objects(copy=False)


In [60]:
#Imputa valores faltantes
df = impute_missing_values(df, method="interpolate")

C:\Users\VAP\Documents\GitHub\ProjetoAplicado-IV\A2\src\utils.py:26: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[num_cols] = df[num_cols].fillna(df[num_cols].mean())


NotImplementedError: IntegerArray does not implement interpolate

In [42]:
# Converte colunas de tipo 'object' para tipos mais específicos
df = df.infer_objects(copy=False)

# Aplica interpolação apenas nas colunas numéricas
num_cols = df.select_dtypes(include=["number"]).columns
df[num_cols] = df[num_cols].astype(float).interpolate(method="linear")
df[num_cols] = df[num_cols].ffill()
df[num_cols] = df[num_cols].bfill()

In [55]:
#Remoção de outliers via z-score
df = remove_outliers_zscore(df, columns=[
    "casos", "rt", "pop", "tempmed", "umidmed", "t_mean", "precip_sum", "pop_density"
])

NameError: name 'remove_outliers_zscore' is not defined

In [44]:
#Criação de variáveis de defasagem (lags) e médias móveis
df.sort_values(["Localidade_id", "year", "week"], inplace=True)

for lag in range(1, 5):
    df[f"casos_lag{lag}"] = df.groupby("Localidade_id")["casos"].shift(lag)

df["casos_roll4"] = df.groupby("Localidade_id")["casos"].rolling(4).mean().reset_index(0, drop=True)


In [45]:
#Normalização das variáveis numéricas
num_cols = ["casos", "Rt", "pop", "tempmed", "umidmed", "t_mean", "precip_sum", "pop_density"] + \
           [f"casos_lag{l}" for l in range(1, 5)] + ["casos_roll4"]

scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


ValueError: Found array with 0 sample(s) (shape=(0, 13)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
#Salva dataset limpo
df_clean = df.dropna()
df_clean.to_csv("data/processed/dataset_clean.csv", index=False)
print("Dataset limpo salvo:", df_clean.shape)

In [32]:
#Arquivo Gerado
#data/processed/dataset_clean.csv: base final para modelagem
